## 분석

In [ ]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-16.0.2"

### 데이터 로드

In [2]:
import pandas as pd
path = '/content/drive/MyDrive/Colab Notebooks/final/'
web = pd.read_csv(path + 'webtoon최종.csv',encoding='cp949')

In [3]:
web[web.illustrator.isna()]

,Unnamed: 0,title,day,writer,illustrator,story,platform,genre,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13


In [4]:
web.head(3)

,Unnamed: 0,title,day,writer,illustrator,story,platform,genre,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,0,참교육,월,채용택,한가람,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다! 부활남 채용...,네이버 웹툰,"스토리, 액션",NaN,NaN,NaN,NaN,NaN,NaN
1,1,신의 탑,월,SIU,SIU,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년 그리고 그런 소년을 시험하는 탑,네이버 웹툰,"스토리, 판타지",NaN,NaN,NaN,NaN,NaN,NaN
2,2,뷰티풀 군바리,월,설이,윤성원,여자도 군대에 간다면? 본격 여자도 군대 가는 만화!,네이버 웹툰,"스토리, 드라마",NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# 컬럼 정리
web=web[['title','writer','illustrator','day','genre','story','platform']]

In [6]:
web.head(3)

,title,writer,illustrator,day,genre,story,platform
0,참교육,채용택,한가람,월,"스토리, 액션",무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다! 부활남 채용...,네이버 웹툰
1,신의 탑,SIU,SIU,월,"스토리, 판타지",자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년 그리고 그런 소년을 시험하는 탑,네이버 웹툰
2,뷰티풀 군바리,설이,윤성원,월,"스토리, 드라마",여자도 군대에 간다면? 본격 여자도 군대 가는 만화!,네이버 웹툰


In [7]:
type(web.title[0])

str

### 전처리

In [8]:
import re

def hangul(text):
    return re.sub('[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣]', ' ', text)

In [9]:
web_copy = web.copy()
# web_copy['title'] = web_copy['title'].apply(lambda x:hangul(x))
web_copy['genre'] = web_copy['genre'].apply(lambda x:hangul(x))
web_copy['story'] = web_copy['story'].apply(lambda x:hangul(x))

web_copy.head(3)

,title,writer,illustrator,day,genre,story,platform
0,참교육,채용택,한가람,월,스토리 액션,무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다 부활남 채용...,네이버 웹툰
1,신의 탑,SIU,SIU,월,스토리 판타지,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년 그리고 그런 소년을 시험하는 탑,네이버 웹툰
2,뷰티풀 군바리,설이,윤성원,월,스토리 드라마,여자도 군대에 간다면 본격 여자도 군대 가는 만화,네이버 웹툰


In [10]:
print(web.iloc[0,5])
print(web_copy.iloc[0,5])

무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다! 부활남 채용택 작가 신석기녀 한가람 작가의 신작!
무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다  부활남 채용택 작가 신석기녀 한가람 작가의 신작 


### 불용어제거

In [11]:
w_copy = web.copy()

In [12]:
title=w_copy.title.unique()

In [13]:
temp=w_copy.writer.apply(lambda x: hangul(x))
writer = temp.unique()

In [14]:
temp = w_copy.illustrator.apply(lambda x: hangul(x))
illustrator = temp.unique()

In [15]:
title=list(title)
writer = list(writer)
illustrator = list(illustrator)

In [16]:
with open(path + 'stopwords_korean.txt','r',encoding='utf8') as f:
    stopwords = f.readline()
stopwords = stopwords.split()
print(stopwords[:5])

['아', '휴', '아이구', '아이쿠', '아이고']


In [17]:
new_word=['신작','은','는','도','고','이다','고','다','못','게','주인공','레알','만화','작가']
for word in [new_word,title,writer,illustrator]:
    stopwords.extend(word)
print(stopwords)

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를', '들면', '예를', '들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수', '없다', '해서는', '안된다', '뿐만', '아니라', '만이', '아니다', '만은', '아니다', '막론하고', '관계없이', '그치지', '않다', '그러나', '그런데', '하지만', '든간에', '논하지', '않다', '따지지', '않다', '설사', '비록', '더라도', '아니면', '만', '못하다', '하는', '편이', '낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이', '외에', '이', '밖에', '하여야', '비로소', '한다면', '몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할', '생각이다', '하려고하다', '이리하여', '그리하여', '그렇게', '함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에', '틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에

### 토큰화

In [18]:
from konlpy.tag import Okt, Hannanum, Kkma, Komoran, Mecab
okt = Okt() # Okt : Open Korean Text
hannanum = Hannanum() # Hannanum(한나눔) : KAIST
kkma = Kkma() # Kkma(꼬꼬마) : 서울대학교
komoran = Komoran() # Komoran : Shineware

def okt_tokenizer(text):
    # 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
    tokens_ko = okt.morphs(text)
    filtered_words = []
    # 각 문장별 단어에 대해 스톱 워드 제거
    for word in tokens_ko:
        if word not in stopwords:
            filtered_words.append(word)
    
    return filtered_words

def kkma_tokenizer(text):
    # 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
    tokens_ko = kkma.morphs(text)
    filtered_words = []
    # 각 문장별 단어에 대해 스톱 워드 제거
    for word in tokens_ko:
        if word not in stopwords:
            filtered_words.append(word)
    
    return filtered_words

def hannanum_tokenizer(text):
    # 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
    tokens_ko = hannanum.morphs(text)
    filtered_words = []
    # 각 문장별 단어에 대해 스톱 워드 제거
    for word in tokens_ko:
        if word not in stopwords:
            filtered_words.append(word)
    
    return filtered_words

def komoran_tokenizer(text):
    # 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
    tokens_ko = komoran.morphs(text)
    filtered_words = []
    # 각 문장별 단어에 대해 스톱 워드 제거
    for word in tokens_ko:
        if word not in stopwords:
            filtered_words.append(word)
    
    return filtered_words

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelBinarizer

# 입력 값 : 토큰화 도구
def get_sparse_matrix(tokenizer):
    # 제목, 장르 벡터화
    # tfidf_nstory_vec = TfidfVectorizer(tokenizer=tokenizer)
    # tfidf_title_matrix = tfidf_nstory_vec.fit_transform(web_copy['title'])
    
    cnt_genre_vec = CountVectorizer()
    cnt_genre_matrix = cnt_genre_vec.fit_transform(web_copy['genre'])

    # 작가 인코딩

    lb = LabelBinarizer(sparse_output=True) # 다른 행렬과 합치기 위하여 희소 행렬로 반환
    lb_writer_matrix = lb.fit_transform(web_copy['writer'])
    lb_illustrator_matrix = lb.fit_transform(web_copy['illustrator'])
    
    # 줄거리 벡터화
    tfidf_story_vec = TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1, 2))
    tfidf_story_matrix = tfidf_story_vec.fit_transform(web_copy['story'])

    # tfidf_title_matrix,
    
    return  cnt_genre_matrix, lb_writer_matrix, lb_illustrator_matrix, tfidf_story_matrix

In [20]:
web_copy.isna().sum()

title          0
writer         0
illustrator    0
day            0
genre          0
story          0
platform       0
dtype: int64

In [21]:
# 결측값 확인
web_copy[web_copy.writer.isna()]

,title,writer,illustrator,day,genre,story,platform


In [22]:
%%time

okt_matrices = get_sparse_matrix(okt_tokenizer)
kkma_matrices = get_sparse_matrix(kkma_tokenizer)
hannanum_matrices = get_sparse_matrix(hannanum_tokenizer)
komoran_matrices = get_sparse_matrix(komoran_tokenizer)

CPU times: user 18min 24s, sys: 7.04 s, total: 18min 31s
Wall time: 10min 38s


In [68]:
import numpy as np

np.save(path+'matrics/'+'okt_matrices',okt_matrices)
np.save(path+'matrics/'+'kkma_matrices',kkma_matrices)
np.save(path+'matrics/'+'hannanum_matrices',hannanum_matrices)
np.save(path+'matrics/'+'komoran_matrices',komoran_matrices)

In [ ]:
matrics_path = '/content/drive/MyDrive/Colab Notebooks/final/matrics'

okt_matrices = np.load(path+'okt_matrices.npy',allow_pickle=True)
kkma_matrices = np.load(path+'kkma_matrices.npy',allow_pickle=True)
hannanum_matrices = np.load(path+'hannanum_matrices.npy',allow_pickle=True)
komoran_matrices = np.load(path+'komoran_matrices.npy',allow_pickle=True)

In [23]:
from scipy.sparse import hstack

def hstack_matrices(matrices, noauthor, w_author, w_genre):
    cnt_genre_matrix, lb_writer_matrix, lb_illustrator_matrix, tfidf_story_matrix = matrices
    
    # tfidf_title_matrix,
    # 만들어진 행렬을 합치기 위한 리스트
    sparse_matrix_list = [cnt_genre_matrix * w_genre, tfidf_story_matrix]
    if not noauthor:
        sparse_matrix_list.append(lb_writer_matrix * w_author)
        sparse_matrix_list.append(lb_illustrator_matrix * w_author)
    
    # if not notitle:
    #     sparse_matrix_list.append(tfidf_title_matrix)
    
    sparse_matrix_list = tuple(sparse_matrix_list)

    # 행렬 결합
    X_features_sparse = hstack(sparse_matrix_list).tocsr()
#     print(type(X_features_sparse), X_features_sparse.shape)
    return X_features_sparse

In [24]:
# 웹툰 제목을 입력하면 해당 인덱스를 반환해줄 Series 생성
indices = pd.Series(web.index, index=web['title'])
indices.head(3)

title
참교육        0
신의 탑       1
뷰티풀 군바리    2
dtype: int64

In [25]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

tokenizer_matrices = [okt_matrices, kkma_matrices, hannanum_matrices, komoran_matrices]
# notitle=False,
def get_recomm(title, rank=10, same_genre=False, noauthor=False,  w_author=0.7, w_genre=1.3, tokenizer_matrices=tokenizer_matrices):
    # 코사인 유사도를 담을 array 생성
    cosine_sim = np.zeros((len(web), len(web)))
    
    # 각 토큰화 도구로 만들어진 행렬을 합쳐서 코사인 유사도 생성
    for matrices in tokenizer_matrices:
        X_features_sparse = hstack_matrices(matrices, noauthor=noauthor,  w_author=w_author, w_genre=w_genre)
        cosine_sim += cosine_similarity(X_features_sparse, X_features_sparse)
    cosine_sim /= len(tokenizer_matrices)
    
    # 결과 출력을 위한 데이터 프레임 복사
    print_copy = web.copy()
    
    # 선택한 웹툰 제목으로부터 해당되는 인덱스를 받아옴
    idx = indices[title]
    
    # 인덱스로부터 장르를 뽑아옴
    genre = print_copy.loc[idx, 'genre']

    # 유사도를 DataFrame에 붙임
    print_copy['similarity'] = cosine_sim[idx]
    
    if same_genre:
        # 동일 장르만을 보여줄 경우
        return print_copy[print_copy['genre'] == genre].sort_values(by='similarity', ascending=False).iloc[1:1+rank, :]
        
    else:
        # 다른 장르도 보여줄 경우
        return print_copy.sort_values(by='similarity', ascending=False).iloc[1:1+rank, :]

## 가중치 테스트

### w_author=0.7, w_genre=1.3

In [26]:
get_recomm('신의 탑')

,title,writer,illustrator,day,genre,story,platform,similarity
261,나 혼자 만렙 뉴비,메슬로우,스윙뱃,금,"스토리, 판타지","게임 너튜버로 활동 중인 주인공 진혁은 유일하게 시련의 탑 의 엔딩을 봤지만, 게임...",네이버 웹툰,0.645246
1441,창위의일루젼,시우,시우,완결,"스토리, 판타지",눈이 오는 겨울밤 소년을 쫓아 오는 악령의 기사...,네이버 웹툰,0.639768
89,빌런투킬,퓨핀,은지,화,"스토리, 판타지",동료를 살해했다는 억울한 누명을 쓰고 사망한 카시안. 모든 게 끝이라고 생각한 순간...,네이버 웹툰,0.638517
931,환상적인 소년,후은,후은,완결,"스토리, 판타지",평범한 소년과 환상적인 소녀의 위험한 동행 이야기.,네이버 웹툰,0.638414
715,골든 체인지,브림스,브림스,완결,"스토리, 판타지",세상에서 가장 불운한 소녀 강유와 소녀의 불운을 막아주는 수상한 산신령 우암이 펼치...,네이버 웹툰,0.637932
60,오로지 오로라,홍달,홍달,월,"스토리, 판타지","마법 소녀가 불법인 세상, 이 세계의 마법 소녀는 사랑받는 존재가 아니다. 한창 잘...",네이버 웹툰,0.637373
1179,붉은 실,동비,동비,완결,"스토리, 판타지",생명의 붉은 실을 두고 벌어지는 사건 그리고 거기에 휘말린 소년과 소녀의 이야기,네이버 웹툰,0.636860
1081,천년구미호,기량,기량,완결,"스토리, 판타지",세습무의 운명을 가진 소녀와 천년의 속박에서 벗어나기 위한 구미호이야기,네이버 웹툰,0.636491
569,리안소울의 엑소클럽,신진오,박희정,완결,"스토리, 판타지",녹안 을 가진 신비한 소녀 소울과 천재 사이코패스 리안. 미스테리한 저주계 사건을 ...,네이버 웹툰,0.636449
510,뫼신 사냥꾼,윤현승,방승현,완결,"스토리, 판타지","신령과도 같은 짐승 뫼신.떠돌이 무사 한세희는 그들을 사냥하고, 그의 앞에 신비한 ...",네이버 웹툰,0.635981


In [27]:
get_recomm('신의 탑',noauthor=True)

,title,writer,illustrator,day,genre,story,platform,similarity
261,나 혼자 만렙 뉴비,메슬로우,스윙뱃,금,"스토리, 판타지","게임 너튜버로 활동 중인 주인공 진혁은 유일하게 시련의 탑 의 엔딩을 봤지만, 게임...",네이버 웹툰,0.789617
1441,창위의일루젼,시우,시우,완결,"스토리, 판타지",눈이 오는 겨울밤 소년을 쫓아 오는 악령의 기사...,네이버 웹툰,0.782912
89,빌런투킬,퓨핀,은지,화,"스토리, 판타지",동료를 살해했다는 억울한 누명을 쓰고 사망한 카시안. 모든 게 끝이라고 생각한 순간...,네이버 웹툰,0.781381
931,환상적인 소년,후은,후은,완결,"스토리, 판타지",평범한 소년과 환상적인 소녀의 위험한 동행 이야기.,네이버 웹툰,0.781256
715,골든 체인지,브림스,브림스,완결,"스토리, 판타지",세상에서 가장 불운한 소녀 강유와 소녀의 불운을 막아주는 수상한 산신령 우암이 펼치...,네이버 웹툰,0.780665
60,오로지 오로라,홍달,홍달,월,"스토리, 판타지","마법 소녀가 불법인 세상, 이 세계의 마법 소녀는 사랑받는 존재가 아니다. 한창 잘...",네이버 웹툰,0.779982
1179,붉은 실,동비,동비,완결,"스토리, 판타지",생명의 붉은 실을 두고 벌어지는 사건 그리고 거기에 휘말린 소년과 소녀의 이야기,네이버 웹툰,0.779354
1081,천년구미호,기량,기량,완결,"스토리, 판타지",세습무의 운명을 가진 소녀와 천년의 속박에서 벗어나기 위한 구미호이야기,네이버 웹툰,0.778903
569,리안소울의 엑소클럽,신진오,박희정,완결,"스토리, 판타지",녹안 을 가진 신비한 소녀 소울과 천재 사이코패스 리안. 미스테리한 저주계 사건을 ...,네이버 웹툰,0.778851
510,뫼신 사냥꾼,윤현승,방승현,완결,"스토리, 판타지","신령과도 같은 짐승 뫼신.떠돌이 무사 한세희는 그들을 사냥하고, 그의 앞에 신비한 ...",네이버 웹툰,0.778279


In [28]:
get_recomm('비흔')

,title,writer,illustrator,day,genre,story,platform,similarity
1455,태왕광개토,청안랑,김인호,완결,"스토리, 시대극",고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.636000
1276,삵의 발톱,손영완,손영완,완결,"스토리, 시대극","중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.635403
1016,천국의 신화 6부,올댓스토리,이현세,완결,"스토리, 시대극","10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.634018
1387,흐드러지다,연제원,연제원,완결,"스토리, 시대극","지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.633802
763,조선방랑야사,가현,가현,완결,"스토리, 시대극",한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.633149
1177,문아,팬마,팬마,완결,"스토리, 시대극",평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.632014
138,화산귀환,비가,LICO,수,"스토리, 시대극",대 화산파 13대 제자.천하삼대검수 매화검존 청명. 천하를 혼란에 빠뜨린 고금제일마...,네이버 웹툰,0.631674
900,왕으로 살다,네스티캣,네스티캣,완결,"스토리, 시대극",살해당한 쌍둥이 오빠를 대신해 왕의 자리에 오른 화현의 운명은?,네이버 웹툰,0.631421
1325,한섬세대,유승진,유승진,완결,"스토리, 시대극","조선후기의 평범한 관리지망생 한섬. 만만치 않은 길이지만, 한섬이는 만만디. 과연 ...",네이버 웹툰,0.631224
1137,"아이소포스 1, 2부",김양수,도가도,완결,"스토리, 시대극","이솝우화의 이솝, 자신의 운명을 바꾼 한 사람의 이야기.",네이버 웹툰,0.631137


In [29]:
get_recomm('비흔',noauthor=True)

,title,writer,illustrator,day,genre,story,platform,similarity
1455,태왕광개토,청안랑,김인호,완결,"스토리, 시대극",고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.778301
1276,삵의 발톱,손영완,손영완,완결,"스토리, 시대극","중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.777571
1016,천국의 신화 6부,올댓스토리,이현세,완결,"스토리, 시대극","10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.775876
1387,흐드러지다,연제원,연제원,완결,"스토리, 시대극","지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.775611
763,조선방랑야사,가현,가현,완결,"스토리, 시대극",한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.774813
1177,문아,팬마,팬마,완결,"스토리, 시대극",평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.773424
138,화산귀환,비가,LICO,수,"스토리, 시대극",대 화산파 13대 제자.천하삼대검수 매화검존 청명. 천하를 혼란에 빠뜨린 고금제일마...,네이버 웹툰,0.773008
900,왕으로 살다,네스티캣,네스티캣,완결,"스토리, 시대극",살해당한 쌍둥이 오빠를 대신해 왕의 자리에 오른 화현의 운명은?,네이버 웹툰,0.772698
1325,한섬세대,유승진,유승진,완결,"스토리, 시대극","조선후기의 평범한 관리지망생 한섬. 만만치 않은 길이지만, 한섬이는 만만디. 과연 ...",네이버 웹툰,0.772457
1137,"아이소포스 1, 2부",김양수,도가도,완결,"스토리, 시대극","이솝우화의 이솝, 자신의 운명을 바꾼 한 사람의 이야기.",네이버 웹툰,0.772351


In [30]:
get_recomm('죽음에 관하여')

,title,writer,illustrator,day,genre,story,platform,similarity
431,굿 리스너,쥬드프라이데이,쥬드프라이데이,일,"옴니버스, 드라마",사무실을 공짜로 빌려쓰게 된 만화가 쥬드씨. 그 사무실은 원래 고민상담소로 쓰이던 ...,네이버 웹툰,0.636557
1411,구름의 노래,호랑,호랑,완결,"옴니버스, 드라마",삶을 풍요롭게 하는 음악 밴드를 통해 삶으로부터 탈출하고자 하는 그들.노래를 통해 ...,네이버 웹툰,0.635378
1124,오민혁 단편선,오민혁,오민혁,완결,"옴니버스, 드라마","한 명의 젊은 작가가 그리는, 때로는 기발하고 때로는 따뜻한 세계! 짧지만 그래서 ...",네이버 웹툰,0.633405
1263,아이들의 권 선생님,호우,호우,완결,"옴니버스, 드라마",아이들의 히어로! 조폭 선생님과 다섯 꼬마들의 소소한 학교 일상 이야기.,네이버 웹툰,0.632377
1408,싸우자귀신아,임인스,임인스,완결,"옴니버스, 드라마",귀신과 퇴마사의 좌충우돌 한판승부! 골 때리는 장르 불명의 이야기!,네이버 웹툰,0.632212
355,단편.zip,웹툰작가,웹툰작가,토,"옴니버스, 드라마","짧고 강렬하고 신선한 단편 모음 . 장르, 소재, 설정, 길이 제한 없음! 매주 새...",네이버 웹툰,0.632198
1164,차차차,한나,한나,완결,"옴니버스, 드라마",감성웹툰 묵회 한나 작가의 신작! 한가한 신생 찻집 고운 다실 에서 일어나는 따뜻하...,네이버 웹툰,0.632197
1255,고고루키루,한경찰,한경찰,완결,"옴니버스, 드라마","정체불명의 3인조, 천진난만 켄, 카리스마 선녀, 쉬크노노 루키루가 만들어내는 황당...",네이버 웹툰,0.631983
1435,사노라면,네이버,작가,완결,"옴니버스, 드라마",사노라면 힘든날도 있습니다.하지만 함께 하는 사람들이 있기에 다시 해가 뜰 날을 기...,네이버 웹툰,0.631980
1205,2014 루키 단편선-지옥...,웹툰작가,웹툰작가,완결,"옴니버스, 드라마","전설적인 작가 이현세, 이두호가 직접 프로듀싱한신인 작가 단편선 네번째 시즌!",네이버 웹툰,0.631948


In [31]:
get_recomm('죽음에 관하여',noauthor=True)

,title,writer,illustrator,day,genre,story,platform,similarity
431,굿 리스너,쥬드프라이데이,쥬드프라이데이,일,"옴니버스, 드라마",사무실을 공짜로 빌려쓰게 된 만화가 쥬드씨. 그 사무실은 원래 고민상담소로 쓰이던 ...,네이버 웹툰,0.778982
1411,구름의 노래,호랑,호랑,완결,"옴니버스, 드라마",삶을 풍요롭게 하는 음악 밴드를 통해 삶으로부터 탈출하고자 하는 그들.노래를 통해 ...,네이버 웹툰,0.777540
1124,오민혁 단편선,오민혁,오민혁,완결,"옴니버스, 드라마","한 명의 젊은 작가가 그리는, 때로는 기발하고 때로는 따뜻한 세계! 짧지만 그래서 ...",네이버 웹툰,0.775125
1263,아이들의 권 선생님,호우,호우,완결,"옴니버스, 드라마",아이들의 히어로! 조폭 선생님과 다섯 꼬마들의 소소한 학교 일상 이야기.,네이버 웹툰,0.773868
1408,싸우자귀신아,임인스,임인스,완결,"옴니버스, 드라마",귀신과 퇴마사의 좌충우돌 한판승부! 골 때리는 장르 불명의 이야기!,네이버 웹툰,0.773665
355,단편.zip,웹툰작가,웹툰작가,토,"옴니버스, 드라마","짧고 강렬하고 신선한 단편 모음 . 장르, 소재, 설정, 길이 제한 없음! 매주 새...",네이버 웹툰,0.773649
1164,차차차,한나,한나,완결,"옴니버스, 드라마",감성웹툰 묵회 한나 작가의 신작! 한가한 신생 찻집 고운 다실 에서 일어나는 따뜻하...,네이버 웹툰,0.773647
1255,고고루키루,한경찰,한경찰,완결,"옴니버스, 드라마","정체불명의 3인조, 천진난만 켄, 카리스마 선녀, 쉬크노노 루키루가 만들어내는 황당...",네이버 웹툰,0.773386
1435,사노라면,네이버,작가,완결,"옴니버스, 드라마",사노라면 힘든날도 있습니다.하지만 함께 하는 사람들이 있기에 다시 해가 뜰 날을 기...,네이버 웹툰,0.773382
1205,2014 루키 단편선-지옥...,웹툰작가,웹툰작가,완결,"옴니버스, 드라마","전설적인 작가 이현세, 이두호가 직접 프로듀싱한신인 작가 단편선 네번째 시즌!",네이버 웹툰,0.773343


In [32]:
get_recomm('마음의소리')

,title,writer,illustrator,day,genre,story,platform,similarity
1454,취업의소리,조석,워니,완결,"에피소드, 개그","조석, 박종원 2인의 작가들이 펼치는,팍팍!! 막힘 없이 뚫리는 취업이야기. 웃음은...",네이버 웹툰,0.722015
129,피로만땅,샤니,샤니,화,"에피소드, 개그",203호 저승사자 샤니 작가 신작. 취준생 유림은 밤 9시에 면접을 보러오라는 연락...,네이버 웹툰,0.634582
22,오늘의 순정망화,손하기,손하기,"월, 화, 목","에피소드, 개그",그마고 초엘리트집단 삼국시대 앞에 나타난 전학생 가야! 가야의 마음을 얻는 건 누구?,네이버 웹툰,0.632967
946,조선왕조실톡,무적핑크,무적핑크,완결,"에피소드, 개그","어느 날 메신저로 찾아온, 조선 시대 그분들의 시시콜콜한 이야기.무적핑크 작가 신작!",네이버 웹툰,0.632387
772,203호 저승사자,샤니,샤니,완결,"에피소드, 개그",어느날 저승사자가 찾아왔다.그런데 이 저승사자... 제 정신이 아니다.,네이버 웹툰,0.631840
1457,트라우마,곽백수,곽백수,완결,"에피소드, 개그",황당하고 골 때리는 반전! 정신적 외상 를 입을 만한 초특급 코믹 펀치! 직접 확인하세요,네이버 웹툰,0.631613
1085,밥풀때기,Dylan,Dylan,완결,"에피소드, 개그",열정이 가득한 밥풀때기 아저씨의 다이내믹한 하루! 글로벌 라인 웹툰에서 연재되고 있...,네이버 웹툰,0.631443
841,백귀야행지,아만,아만,완결,"에피소드, 개그",소름과 웃음이 동시에! 빨간 도깨비가 풀어놓는 기이한 이야기글로벌 라인 웹툰에서 연...,네이버 웹툰,0.631413
903,놓지마 정신줄 시즌2,신태훈,나승훈,완결,"에피소드, 개그","정신, 정주리 남매와 독특한 주변 인물들의정신줄 놓게 만드는 다양한 에피소드들!본격...",네이버 웹툰,0.631369
124,고등매직,잭형,잭형,화,"에피소드, 개그",평범한 고등학생인 마삼룡 의 집에 갑자기 삼촌이 찾아온다. 삼룡의 가족과 함께 지내...,네이버 웹툰,0.631118


In [33]:
get_recomm('마음의소리', noauthor=True)

,title,writer,illustrator,day,genre,story,platform,similarity
129,피로만땅,샤니,샤니,화,"에피소드, 개그",203호 저승사자 샤니 작가 신작. 취준생 유림은 밤 9시에 면접을 보러오라는 연락...,네이버 웹툰,0.776566
22,오늘의 순정망화,손하기,손하기,"월, 화, 목","에피소드, 개그",그마고 초엘리트집단 삼국시대 앞에 나타난 전학생 가야! 가야의 마음을 얻는 건 누구?,네이버 웹툰,0.774590
946,조선왕조실톡,무적핑크,무적핑크,완결,"에피소드, 개그","어느 날 메신저로 찾아온, 조선 시대 그분들의 시시콜콜한 이야기.무적핑크 작가 신작!",네이버 웹툰,0.773880
772,203호 저승사자,샤니,샤니,완결,"에피소드, 개그",어느날 저승사자가 찾아왔다.그런데 이 저승사자... 제 정신이 아니다.,네이버 웹툰,0.773211
1457,트라우마,곽백수,곽백수,완결,"에피소드, 개그",황당하고 골 때리는 반전! 정신적 외상 를 입을 만한 초특급 코믹 펀치! 직접 확인하세요,네이버 웹툰,0.772933
1085,밥풀때기,Dylan,Dylan,완결,"에피소드, 개그",열정이 가득한 밥풀때기 아저씨의 다이내믹한 하루! 글로벌 라인 웹툰에서 연재되고 있...,네이버 웹툰,0.772725
841,백귀야행지,아만,아만,완결,"에피소드, 개그",소름과 웃음이 동시에! 빨간 도깨비가 풀어놓는 기이한 이야기글로벌 라인 웹툰에서 연...,네이버 웹툰,0.772688
903,놓지마 정신줄 시즌2,신태훈,나승훈,완결,"에피소드, 개그","정신, 정주리 남매와 독특한 주변 인물들의정신줄 놓게 만드는 다양한 에피소드들!본격...",네이버 웹툰,0.772635
124,고등매직,잭형,잭형,화,"에피소드, 개그",평범한 고등학생인 마삼룡 의 집에 갑자기 삼촌이 찾아온다. 삼룡의 가족과 함께 지내...,네이버 웹툰,0.772327
1254,내일은 웹툰,신의철,신의철,완결,"에피소드, 개그","내일 나오는 만화를 그리며, 언제나 내일을 사는 웹툰 작가들의 만화같은 이야기.",네이버 웹툰,0.772024


### w_author=1.3, w_genre=1


In [34]:
get_recomm('신의 탑',w_author=1.3, w_genre=1)

,title,writer,illustrator,day,genre,story,platform,similarity
261,나 혼자 만렙 뉴비,메슬로우,스윙뱃,금,"스토리, 판타지","게임 너튜버로 활동 중인 주인공 진혁은 유일하게 시련의 탑 의 엔딩을 봤지만, 게임...",네이버 웹툰,0.325787
1441,창위의일루젼,시우,시우,완결,"스토리, 판타지",눈이 오는 겨울밤 소년을 쫓아 오는 악령의 기사...,네이버 웹툰,0.321184
89,빌런투킬,퓨핀,은지,화,"스토리, 판타지",동료를 살해했다는 억울한 누명을 쓰고 사망한 카시안. 모든 게 끝이라고 생각한 순간...,네이버 웹툰,0.320133
931,환상적인 소년,후은,후은,완결,"스토리, 판타지",평범한 소년과 환상적인 소녀의 위험한 동행 이야기.,네이버 웹툰,0.320047
715,골든 체인지,브림스,브림스,완결,"스토리, 판타지",세상에서 가장 불운한 소녀 강유와 소녀의 불운을 막아주는 수상한 산신령 우암이 펼치...,네이버 웹툰,0.319642
60,오로지 오로라,홍달,홍달,월,"스토리, 판타지","마법 소녀가 불법인 세상, 이 세계의 마법 소녀는 사랑받는 존재가 아니다. 한창 잘...",네이버 웹툰,0.319172
1179,붉은 실,동비,동비,완결,"스토리, 판타지",생명의 붉은 실을 두고 벌어지는 사건 그리고 거기에 휘말린 소년과 소녀의 이야기,네이버 웹툰,0.318741
1081,천년구미호,기량,기량,완결,"스토리, 판타지",세습무의 운명을 가진 소녀와 천년의 속박에서 벗어나기 위한 구미호이야기,네이버 웹툰,0.318432
569,리안소울의 엑소클럽,신진오,박희정,완결,"스토리, 판타지",녹안 을 가진 신비한 소녀 소울과 천재 사이코패스 리안. 미스테리한 저주계 사건을 ...,네이버 웹툰,0.318396
510,뫼신 사냥꾼,윤현승,방승현,완결,"스토리, 판타지","신령과도 같은 짐승 뫼신.떠돌이 무사 한세희는 그들을 사냥하고, 그의 앞에 신비한 ...",네이버 웹툰,0.318003


In [35]:
get_recomm('신의 탑',noauthor=True,w_author=1.3, w_genre=1)

,title,writer,illustrator,day,genre,story,platform,similarity
261,나 혼자 만렙 뉴비,메슬로우,스윙뱃,금,"스토리, 판타지","게임 너튜버로 활동 중인 주인공 진혁은 유일하게 시련의 탑 의 엔딩을 봤지만, 게임...",네이버 웹툰,0.692840
1441,창위의일루젼,시우,시우,완결,"스토리, 판타지",눈이 오는 겨울밤 소년을 쫓아 오는 악령의 기사...,네이버 웹툰,0.683052
89,빌런투킬,퓨핀,은지,화,"스토리, 판타지",동료를 살해했다는 억울한 누명을 쓰고 사망한 카시안. 모든 게 끝이라고 생각한 순간...,네이버 웹툰,0.680817
931,환상적인 소년,후은,후은,완결,"스토리, 판타지",평범한 소년과 환상적인 소녀의 위험한 동행 이야기.,네이버 웹툰,0.680634
715,골든 체인지,브림스,브림스,완결,"스토리, 판타지",세상에서 가장 불운한 소녀 강유와 소녀의 불운을 막아주는 수상한 산신령 우암이 펼치...,네이버 웹툰,0.679771
60,오로지 오로라,홍달,홍달,월,"스토리, 판타지","마법 소녀가 불법인 세상, 이 세계의 마법 소녀는 사랑받는 존재가 아니다. 한창 잘...",네이버 웹툰,0.678773
1179,붉은 실,동비,동비,완결,"스토리, 판타지",생명의 붉은 실을 두고 벌어지는 사건 그리고 거기에 휘말린 소년과 소녀의 이야기,네이버 웹툰,0.677857
1081,천년구미호,기량,기량,완결,"스토리, 판타지",세습무의 운명을 가진 소녀와 천년의 속박에서 벗어나기 위한 구미호이야기,네이버 웹툰,0.677198
569,리안소울의 엑소클럽,신진오,박희정,완결,"스토리, 판타지",녹안 을 가진 신비한 소녀 소울과 천재 사이코패스 리안. 미스테리한 저주계 사건을 ...,네이버 웹툰,0.677122
510,뫼신 사냥꾼,윤현승,방승현,완결,"스토리, 판타지","신령과도 같은 짐승 뫼신.떠돌이 무사 한세희는 그들을 사냥하고, 그의 앞에 신비한 ...",네이버 웹툰,0.676287


In [59]:
get_recomm('비흔',w_author=1.3, w_genre=1)

,title,writer,illustrator,day,genre,story,platform,similarity
1456,제로,정재한,이나루,완결,"스토리, 드라마","늘 행복한 길만 걸어가라고 지어 준 이름 제로 세상에 남은 그의 유일한 행복, 서연...",네이버 웹툰,0.425198
760,[드라마원작] 스위트홈,김칸비,황영찬,완결,"스토리, 스릴러","절망의 세상을 비관하고 스스로를 고립시킨 한 소년,하지만 이제 소년은 밖으로 나가고...",네이버 웹툰,0.422023
1291,그린보이,정재한,임진국,완결,"스토리, 스포츠","챔피언을 노리는 거침없는 여자, 재인! 그녀를 동경하는 순둥이 남자, 태진! 챔피언...",네이버 웹툰,0.422009
1099,후레자식,김칸비,황영찬,완결,"스토리, 스릴러",내 집안에 살인마가 산다. 본격 세대갈등 잔혹 스릴러,네이버 웹툰,0.421848
1455,태왕광개토,청안랑,김인호,완결,"스토리, 시대극",고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.318019
1276,삵의 발톱,손영완,손영완,완결,"스토리, 시대극","중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.317518
1016,천국의 신화 6부,올댓스토리,이현세,완결,"스토리, 시대극","10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.316354
1387,흐드러지다,연제원,연제원,완결,"스토리, 시대극","지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.316172
763,조선방랑야사,가현,가현,완결,"스토리, 시대극",한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.315624
1177,문아,팬마,팬마,완결,"스토리, 시대극",평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.314670


In [60]:
get_recomm('비흔',noauthor=True,w_author=1.3, w_genre=1)

,title,writer,illustrator,day,genre,story,platform,similarity
1455,태왕광개토,청안랑,김인호,완결,"스토리, 시대극",고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.676319
1276,삵의 발톱,손영완,손영완,완결,"스토리, 시대극","중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.675254
1016,천국의 신화 6부,올댓스토리,이현세,완결,"스토리, 시대극","10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.672779
1387,흐드러지다,연제원,연제원,완결,"스토리, 시대극","지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.672392
763,조선방랑야사,가현,가현,완결,"스토리, 시대극",한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.671227
1177,문아,팬마,팬마,완결,"스토리, 시대극",평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.669199
138,화산귀환,비가,LICO,수,"스토리, 시대극",대 화산파 13대 제자.천하삼대검수 매화검존 청명. 천하를 혼란에 빠뜨린 고금제일마...,네이버 웹툰,0.668591
900,왕으로 살다,네스티캣,네스티캣,완결,"스토리, 시대극",살해당한 쌍둥이 오빠를 대신해 왕의 자리에 오른 화현의 운명은?,네이버 웹툰,0.668139
1325,한섬세대,유승진,유승진,완결,"스토리, 시대극","조선후기의 평범한 관리지망생 한섬. 만만치 않은 길이지만, 한섬이는 만만디. 과연 ...",네이버 웹툰,0.667787
1137,"아이소포스 1, 2부",김양수,도가도,완결,"스토리, 시대극","이솝우화의 이솝, 자신의 운명을 바꾼 한 사람의 이야기.",네이버 웹툰,0.667632


In [61]:
get_recomm('마음의소리',w_author=1.3, w_genre=1)

,title,writer,illustrator,day,genre,story,platform,similarity
1157,조석축구만화,조석,조석,완결,"에피소드, 스포츠",조석 작가가 들려주는 너무나도 솔직담백한 해외축구이야기.,네이버 웹툰,0.703764
1454,취업의소리,조석,워니,완결,"에피소드, 개그","조석, 박종원 2인의 작가들이 펼치는,팍팍!! 막힘 없이 뚫리는 취업이야기. 웃음은...",네이버 웹툰,0.578370
1024,문유,조석,조석,완결,"스토리, 판타지",지구가 멸망했다. 나는 혼자 달에 있다. 마음의 소리 조석의 우주를 넘나드는 판타지...,네이버 웹툰,0.555230
899,조의 영역,조석,조석,완결,"스토리, 스릴러",물에 살던 녀석들의 모습이 상식을 벗어나기 시작했다. 마음의 소리 조석의 재난 스릴러,네이버 웹툰,0.547808
200,묵시의 인플루언서,조석,조석,목,"스토리, 스릴러","어느 날 동네에 생긴 정체 모를 알 하나. 알은 빠른 속도로 퍼지더니, 급기야 알에...",네이버 웹툰,0.530947
1444,N의등대-busted,조석,조석,완결,"스토리, 스릴러","동일한 세계의 색다른 4가지 이야기. 함께 볼 수도, 따로 볼 수도 있는 멀티 플롯...",네이버 웹툰,0.530221
462,행성인간,조석,조석,완결,"스토리, 스릴러",내가 어떤 존재들의 행성이라면?조석 작가가 그리는 신개념 스릴러,네이버 웹툰,0.529781
1331,자율공상축구탐구만화,조석,조석,완결,"옴니버스, 스포츠","월드컵, 유럽리그, 국내리그 등 축구라면 가리지 않고 모두 그려보는 자율축구탐구만화...",네이버 웹툰,0.529781
594,후기,조석,조석,완결,"스토리, 스릴러",무가지에 연재되던 삼류 소설이 사실은 누군가의 살인 후기였다!,네이버 웹툰,0.529781
129,피로만땅,샤니,샤니,화,"에피소드, 개그",203호 저승사자 샤니 작가 신작. 취준생 유림은 밤 9시에 면접을 보러오라는 연락...,네이버 웹툰,0.316827


In [62]:
get_recomm('마음의소리', noauthor=True,w_author=1.3, w_genre=1)

,title,writer,illustrator,day,genre,story,platform,similarity
129,피로만땅,샤니,샤니,화,"에피소드, 개그",203호 저승사자 샤니 작가 신작. 취준생 유림은 밤 9시에 면접을 보러오라는 연락...,네이버 웹툰,0.673786
22,오늘의 순정망화,손하기,손하기,"월, 화, 목","에피소드, 개그",그마고 초엘리트집단 삼국시대 앞에 나타난 전학생 가야! 가야의 마음을 얻는 건 누구?,네이버 웹툰,0.670901
946,조선왕조실톡,무적핑크,무적핑크,완결,"에피소드, 개그","어느 날 메신저로 찾아온, 조선 시대 그분들의 시시콜콜한 이야기.무적핑크 작가 신작!",네이버 웹툰,0.669865
772,203호 저승사자,샤니,샤니,완결,"에피소드, 개그",어느날 저승사자가 찾아왔다.그런데 이 저승사자... 제 정신이 아니다.,네이버 웹툰,0.668888
1457,트라우마,곽백수,곽백수,완결,"에피소드, 개그",황당하고 골 때리는 반전! 정신적 외상 를 입을 만한 초특급 코믹 펀치! 직접 확인하세요,네이버 웹툰,0.668483
1085,밥풀때기,Dylan,Dylan,완결,"에피소드, 개그",열정이 가득한 밥풀때기 아저씨의 다이내믹한 하루! 글로벌 라인 웹툰에서 연재되고 있...,네이버 웹툰,0.668178
841,백귀야행지,아만,아만,완결,"에피소드, 개그",소름과 웃음이 동시에! 빨간 도깨비가 풀어놓는 기이한 이야기글로벌 라인 웹툰에서 연...,네이버 웹툰,0.668125
903,놓지마 정신줄 시즌2,신태훈,나승훈,완결,"에피소드, 개그","정신, 정주리 남매와 독특한 주변 인물들의정신줄 놓게 만드는 다양한 에피소드들!본격...",네이버 웹툰,0.668046
124,고등매직,잭형,잭형,화,"에피소드, 개그",평범한 고등학생인 마삼룡 의 집에 갑자기 삼촌이 찾아온다. 삼룡의 가족과 함께 지내...,네이버 웹툰,0.667598
1254,내일은 웹툰,신의철,신의철,완결,"에피소드, 개그","내일 나오는 만화를 그리며, 언제나 내일을 사는 웹툰 작가들의 만화같은 이야기.",네이버 웹툰,0.667155


In [63]:
get_recomm('죽음에 관하여',w_author=1.3, w_genre=1)

,title,writer,illustrator,day,genre,story,platform,similarity
1233,네가 없는 세상,시니,혀노,완결,"스토리, 드라마",죽음에 관하여 시니 혀노 작가 신작. 나 만 아는 세상.,네이버 웹툰,0.689262
1145,남과 여,혀노,혀노,완결,"스토리, 드라마","죽음에 관하여 , 네가 없는 세상 혀노 작가의 솔로 데뷔작,솔직한 20대의 흔한 사...",네이버 웹툰,0.425213
1123,아이덴티티,시니,수훈,완결,"스토리, 드라마","잠에 들면 다른 사람이 되는 남자 헨리 모리스, 그리고 그 병을 치료하는 .와이번....",네이버 웹툰,0.423416
405,별이삼샵,혀노,혀노,일,"스토리, 드라마",나니? 발신자 제한번호로 그녀에게 마음을 전했던 이야기. 남과여 혀노 작가가 담아낸...,네이버 웹툰,0.423269
264,1초,시니,광운,금,"스토리, 드라마","구조율 100 의 전설적인 소방관. 그의 특별한 능력은 긴장하는 순간, 미래가 보인...",네이버 웹툰,0.422845
431,굿 리스너,쥬드프라이데이,쥬드프라이데이,일,"옴니버스, 드라마",사무실을 공짜로 빌려쓰게 된 만화가 쥬드씨. 그 사무실은 원래 고민상담소로 쓰이던 ...,네이버 웹툰,0.318486
1411,구름의 노래,호랑,호랑,완결,"옴니버스, 드라마",삶을 풍요롭게 하는 음악 밴드를 통해 삶으로부터 탈출하고자 하는 그들.노래를 통해 ...,네이버 웹툰,0.317496
1124,오민혁 단편선,오민혁,오민혁,완결,"옴니버스, 드라마","한 명의 젊은 작가가 그리는, 때로는 기발하고 때로는 따뜻한 세계! 짧지만 그래서 ...",네이버 웹툰,0.315838
1263,아이들의 권 선생님,호우,호우,완결,"옴니버스, 드라마",아이들의 히어로! 조폭 선생님과 다섯 꼬마들의 소소한 학교 일상 이야기.,네이버 웹툰,0.314975
1408,싸우자귀신아,임인스,임인스,완결,"옴니버스, 드라마",귀신과 퇴마사의 좌충우돌 한판승부! 골 때리는 장르 불명의 이야기!,네이버 웹툰,0.314836


In [64]:
get_recomm('죽음에 관하여',noauthor=True,w_author=1.3, w_genre=1)

,title,writer,illustrator,day,genre,story,platform,similarity
431,굿 리스너,쥬드프라이데이,쥬드프라이데이,일,"옴니버스, 드라마",사무실을 공짜로 빌려쓰게 된 만화가 쥬드씨. 그 사무실은 원래 고민상담소로 쓰이던 ...,네이버 웹툰,0.677314
1411,구름의 노래,호랑,호랑,완결,"옴니버스, 드라마",삶을 풍요롭게 하는 음악 밴드를 통해 삶으로부터 탈출하고자 하는 그들.노래를 통해 ...,네이버 웹툰,0.675208
1124,오민혁 단편선,오민혁,오민혁,완결,"옴니버스, 드라마","한 명의 젊은 작가가 그리는, 때로는 기발하고 때로는 따뜻한 세계! 짧지만 그래서 ...",네이버 웹툰,0.671683
1263,아이들의 권 선생님,호우,호우,완결,"옴니버스, 드라마",아이들의 히어로! 조폭 선생님과 다섯 꼬마들의 소소한 학교 일상 이야기.,네이버 웹툰,0.669847
1408,싸우자귀신아,임인스,임인스,완결,"옴니버스, 드라마",귀신과 퇴마사의 좌충우돌 한판승부! 골 때리는 장르 불명의 이야기!,네이버 웹툰,0.669552
355,단편.zip,웹툰작가,웹툰작가,토,"옴니버스, 드라마","짧고 강렬하고 신선한 단편 모음 . 장르, 소재, 설정, 길이 제한 없음! 매주 새...",네이버 웹툰,0.669527
1164,차차차,한나,한나,완결,"옴니버스, 드라마",감성웹툰 묵회 한나 작가의 신작! 한가한 신생 찻집 고운 다실 에서 일어나는 따뜻하...,네이버 웹툰,0.669525
1255,고고루키루,한경찰,한경찰,완결,"옴니버스, 드라마","정체불명의 3인조, 천진난만 켄, 카리스마 선녀, 쉬크노노 루키루가 만들어내는 황당...",네이버 웹툰,0.669143
1435,사노라면,네이버,작가,완결,"옴니버스, 드라마",사노라면 힘든날도 있습니다.하지만 함께 하는 사람들이 있기에 다시 해가 뜰 날을 기...,네이버 웹툰,0.669137
1205,2014 루키 단편선-지옥...,웹툰작가,웹툰작가,완결,"옴니버스, 드라마","전설적인 작가 이현세, 이두호가 직접 프로듀싱한신인 작가 단편선 네번째 시즌!",네이버 웹툰,0.669081


### w_author=1.3, w_genre=0.7

In [42]:
get_recomm('신의 탑',w_author=1.3, w_genre=0.7)

,title,writer,illustrator,day,genre,story,platform,similarity
261,나 혼자 만렙 뉴비,메슬로우,스윙뱃,금,"스토리, 판타지","게임 너튜버로 활동 중인 주인공 진혁은 유일하게 시련의 탑 의 엔딩을 봤지만, 게임...",네이버 웹툰,0.197485
1441,창위의일루젼,시우,시우,완결,"스토리, 판타지",눈이 오는 겨울밤 소년을 쫓아 오는 악령의 기사...,네이버 웹툰,0.192007
89,빌런투킬,퓨핀,은지,화,"스토리, 판타지",동료를 살해했다는 억울한 누명을 쓰고 사망한 카시안. 모든 게 끝이라고 생각한 순간...,네이버 웹툰,0.190756
931,환상적인 소년,후은,후은,완결,"스토리, 판타지",평범한 소년과 환상적인 소녀의 위험한 동행 이야기.,네이버 웹툰,0.190653
715,골든 체인지,브림스,브림스,완결,"스토리, 판타지",세상에서 가장 불운한 소녀 강유와 소녀의 불운을 막아주는 수상한 산신령 우암이 펼치...,네이버 웹툰,0.190170
60,오로지 오로라,홍달,홍달,월,"스토리, 판타지","마법 소녀가 불법인 세상, 이 세계의 마법 소녀는 사랑받는 존재가 아니다. 한창 잘...",네이버 웹툰,0.189612
1179,붉은 실,동비,동비,완결,"스토리, 판타지",생명의 붉은 실을 두고 벌어지는 사건 그리고 거기에 휘말린 소년과 소녀의 이야기,네이버 웹툰,0.189099
1081,천년구미호,기량,기량,완결,"스토리, 판타지",세습무의 운명을 가진 소녀와 천년의 속박에서 벗어나기 위한 구미호이야기,네이버 웹툰,0.188730
569,리안소울의 엑소클럽,신진오,박희정,완결,"스토리, 판타지",녹안 을 가진 신비한 소녀 소울과 천재 사이코패스 리안. 미스테리한 저주계 사건을 ...,네이버 웹툰,0.188688
510,뫼신 사냥꾼,윤현승,방승현,완결,"스토리, 판타지","신령과도 같은 짐승 뫼신.떠돌이 무사 한세희는 그들을 사냥하고, 그의 앞에 신비한 ...",네이버 웹툰,0.188220


In [43]:
get_recomm('신의 탑',noauthor=True,w_author=1.3, w_genre=0.7)

,title,writer,illustrator,day,genre,story,platform,similarity
261,나 혼자 만렙 뉴비,메슬로우,스윙뱃,금,"스토리, 판타지","게임 너튜버로 활동 중인 주인공 진혁은 유일하게 시련의 탑 의 엔딩을 봤지만, 게임...",네이버 웹툰,0.534606
1441,창위의일루젼,시우,시우,완결,"스토리, 판타지",눈이 오는 겨울밤 소년을 쫓아 오는 악령의 기사...,네이버 웹툰,0.519776
89,빌런투킬,퓨핀,은지,화,"스토리, 판타지",동료를 살해했다는 억울한 누명을 쓰고 사망한 카시안. 모든 게 끝이라고 생각한 순간...,네이버 웹툰,0.516389
931,환상적인 소년,후은,후은,완결,"스토리, 판타지",평범한 소년과 환상적인 소녀의 위험한 동행 이야기.,네이버 웹툰,0.516111
715,골든 체인지,브림스,브림스,완결,"스토리, 판타지",세상에서 가장 불운한 소녀 강유와 소녀의 불운을 막아주는 수상한 산신령 우암이 펼치...,네이버 웹툰,0.514805
60,오로지 오로라,홍달,홍달,월,"스토리, 판타지","마법 소녀가 불법인 세상, 이 세계의 마법 소녀는 사랑받는 존재가 아니다. 한창 잘...",네이버 웹툰,0.513293
1179,붉은 실,동비,동비,완결,"스토리, 판타지",생명의 붉은 실을 두고 벌어지는 사건 그리고 거기에 휘말린 소년과 소녀의 이야기,네이버 웹툰,0.511904
1081,천년구미호,기량,기량,완결,"스토리, 판타지",세습무의 운명을 가진 소녀와 천년의 속박에서 벗어나기 위한 구미호이야기,네이버 웹툰,0.510906
569,리안소울의 엑소클럽,신진오,박희정,완결,"스토리, 판타지",녹안 을 가진 신비한 소녀 소울과 천재 사이코패스 리안. 미스테리한 저주계 사건을 ...,네이버 웹툰,0.510791
510,뫼신 사냥꾼,윤현승,방승현,완결,"스토리, 판타지","신령과도 같은 짐승 뫼신.떠돌이 무사 한세희는 그들을 사냥하고, 그의 앞에 신비한 ...",네이버 웹툰,0.509525


In [44]:
get_recomm('비흔',w_author=1.3, w_genre=0.7)

,title,writer,illustrator,day,genre,story,platform,similarity
1456,제로,정재한,이나루,완결,"스토리, 드라마","늘 행복한 길만 걸어가라고 지어 준 이름 제로 세상에 남은 그의 유일한 행복, 서연...",네이버 웹툰,0.410963
760,[드라마원작] 스위트홈,김칸비,황영찬,완결,"스토리, 스릴러","절망의 세상을 비관하고 스스로를 고립시킨 한 소년,하지만 이제 소년은 밖으로 나가고...",네이버 웹툰,0.407184
1291,그린보이,정재한,임진국,완결,"스토리, 스포츠","챔피언을 노리는 거침없는 여자, 재인! 그녀를 동경하는 순둥이 남자, 태진! 챔피언...",네이버 웹툰,0.407167
1099,후레자식,김칸비,황영찬,완결,"스토리, 스릴러",내 집안에 살인마가 산다. 본격 세대갈등 잔혹 스릴러,네이버 웹툰,0.406976
1455,태왕광개토,청안랑,김인호,완결,"스토리, 시대극",고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.188239
1276,삵의 발톱,손영완,손영완,완결,"스토리, 시대극","중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.187642
1016,천국의 신화 6부,올댓스토리,이현세,완결,"스토리, 시대극","10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.186257
1387,흐드러지다,연제원,연제원,완결,"스토리, 시대극","지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.186040
763,조선방랑야사,가현,가현,완결,"스토리, 시대극",한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.185388
1177,문아,팬마,팬마,완결,"스토리, 시대극",평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.184253


In [45]:
get_recomm('비흔',noauthor=True,w_author=1.3, w_genre=0.7)

,title,writer,illustrator,day,genre,story,platform,similarity
1455,태왕광개토,청안랑,김인호,완결,"스토리, 시대극",고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.509575
1276,삵의 발톱,손영완,손영완,완결,"스토리, 시대극","중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.507961
1016,천국의 신화 6부,올댓스토리,이현세,완결,"스토리, 시대극","10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.504211
1387,흐드러지다,연제원,연제원,완결,"스토리, 시대극","지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.503624
763,조선방랑야사,가현,가현,완결,"스토리, 시대극",한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.501859
1177,문아,팬마,팬마,완결,"스토리, 시대극",평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.498787
138,화산귀환,비가,LICO,수,"스토리, 시대극",대 화산파 13대 제자.천하삼대검수 매화검존 청명. 천하를 혼란에 빠뜨린 고금제일마...,네이버 웹툰,0.497865
900,왕으로 살다,네스티캣,네스티캣,완결,"스토리, 시대극",살해당한 쌍둥이 오빠를 대신해 왕의 자리에 오른 화현의 운명은?,네이버 웹툰,0.497180
1325,한섬세대,유승진,유승진,완결,"스토리, 시대극","조선후기의 평범한 관리지망생 한섬. 만만치 않은 길이지만, 한섬이는 만만디. 과연 ...",네이버 웹툰,0.496647
1137,"아이소포스 1, 2부",김양수,도가도,완결,"스토리, 시대극","이솝우화의 이솝, 자신의 운명을 바꾼 한 사람의 이야기.",네이버 웹툰,0.496412


In [46]:
get_recomm('마음의소리',w_author=1.3, w_genre=0.7)

,title,writer,illustrator,day,genre,story,platform,similarity
1157,조석축구만화,조석,조석,완결,"에피소드, 스포츠",조석 작가가 들려주는 너무나도 솔직담백한 해외축구이야기.,네이버 웹툰,0.742540
1024,문유,조석,조석,완결,"스토리, 판타지",지구가 멸망했다. 나는 혼자 달에 있다. 마음의 소리 조석의 우주를 넘나드는 판타지...,네이버 웹툰,0.660889
899,조의 영역,조석,조석,완결,"스토리, 스릴러",물에 살던 녀석들의 모습이 상식을 벗어나기 시작했다. 마음의 소리 조석의 재난 스릴러,네이버 웹툰,0.652055
200,묵시의 인플루언서,조석,조석,목,"스토리, 스릴러","어느 날 동네에 생긴 정체 모를 알 하나. 알은 빠른 속도로 퍼지더니, 급기야 알에...",네이버 웹툰,0.631985
1444,N의등대-busted,조석,조석,완결,"스토리, 스릴러","동일한 세계의 색다른 4가지 이야기. 함께 볼 수도, 따로 볼 수도 있는 멀티 플롯...",네이버 웹툰,0.631121
1331,자율공상축구탐구만화,조석,조석,완결,"옴니버스, 스포츠","월드컵, 유럽리그, 국내리그 등 축구라면 가리지 않고 모두 그려보는 자율축구탐구만화...",네이버 웹툰,0.630597
594,후기,조석,조석,완결,"스토리, 스릴러",무가지에 연재되던 삼류 소설이 사실은 누군가의 살인 후기였다!,네이버 웹툰,0.630597
462,행성인간,조석,조석,완결,"스토리, 스릴러",내가 어떤 존재들의 행성이라면?조석 작가가 그리는 신개념 스릴러,네이버 웹툰,0.630597
1454,취업의소리,조석,워니,완결,"에피소드, 개그","조석, 박종원 2인의 작가들이 펼치는,팍팍!! 막힘 없이 뚫리는 취업이야기. 웃음은...",네이버 웹툰,0.498134
129,피로만땅,샤니,샤니,화,"에피소드, 개그",203호 저승사자 샤니 작가 신작. 취준생 유림은 밤 9시에 면접을 보러오라는 연락...,네이버 웹툰,0.186820


In [47]:
get_recomm('마음의소리', noauthor=True,w_author=1.3, w_genre=0.7)

,title,writer,illustrator,day,genre,story,platform,similarity
129,피로만땅,샤니,샤니,화,"에피소드, 개그",203호 저승사자 샤니 작가 신작. 취준생 유림은 밤 9시에 면접을 보러오라는 연락...,네이버 웹툰,0.505736
22,오늘의 순정망화,손하기,손하기,"월, 화, 목","에피소드, 개그",그마고 초엘리트집단 삼국시대 앞에 나타난 전학생 가야! 가야의 마음을 얻는 건 누구?,네이버 웹툰,0.501366
946,조선왕조실톡,무적핑크,무적핑크,완결,"에피소드, 개그","어느 날 메신저로 찾아온, 조선 시대 그분들의 시시콜콜한 이야기.무적핑크 작가 신작!",네이버 웹툰,0.499796
772,203호 저승사자,샤니,샤니,완결,"에피소드, 개그",어느날 저승사자가 찾아왔다.그런데 이 저승사자... 제 정신이 아니다.,네이버 웹툰,0.498315
1457,트라우마,곽백수,곽백수,완결,"에피소드, 개그",황당하고 골 때리는 반전! 정신적 외상 를 입을 만한 초특급 코믹 펀치! 직접 확인하세요,네이버 웹툰,0.497701
1085,밥풀때기,Dylan,Dylan,완결,"에피소드, 개그",열정이 가득한 밥풀때기 아저씨의 다이내믹한 하루! 글로벌 라인 웹툰에서 연재되고 있...,네이버 웹툰,0.497240
841,백귀야행지,아만,아만,완결,"에피소드, 개그",소름과 웃음이 동시에! 빨간 도깨비가 풀어놓는 기이한 이야기글로벌 라인 웹툰에서 연...,네이버 웹툰,0.497159
903,놓지마 정신줄 시즌2,신태훈,나승훈,완결,"에피소드, 개그","정신, 정주리 남매와 독특한 주변 인물들의정신줄 놓게 만드는 다양한 에피소드들!본격...",네이버 웹툰,0.497040
124,고등매직,잭형,잭형,화,"에피소드, 개그",평범한 고등학생인 마삼룡 의 집에 갑자기 삼촌이 찾아온다. 삼룡의 가족과 함께 지내...,네이버 웹툰,0.496361
1254,내일은 웹툰,신의철,신의철,완결,"에피소드, 개그","내일 나오는 만화를 그리며, 언제나 내일을 사는 웹툰 작가들의 만화같은 이야기.",네이버 웹툰,0.495689


In [48]:
get_recomm('죽음에 관하여',w_author=1.3, w_genre=0.7)

,title,writer,illustrator,day,genre,story,platform,similarity
1233,네가 없는 세상,시니,혀노,완결,"스토리, 드라마",죽음에 관하여 시니 혀노 작가 신작. 나 만 아는 세상.,네이버 웹툰,0.725279
1145,남과 여,혀노,혀노,완결,"스토리, 드라마","죽음에 관하여 , 네가 없는 세상 혀노 작가의 솔로 데뷔작,솔직한 20대의 흔한 사...",네이버 웹툰,0.410981
1123,아이덴티티,시니,수훈,완결,"스토리, 드라마","잠에 들면 다른 사람이 되는 남자 헨리 모리스, 그리고 그 병을 치료하는 .와이번....",네이버 웹툰,0.408843
405,별이삼샵,혀노,혀노,일,"스토리, 드라마",나니? 발신자 제한번호로 그녀에게 마음을 전했던 이야기. 남과여 혀노 작가가 담아낸...,네이버 웹툰,0.408668
264,1초,시니,광운,금,"스토리, 드라마","구조율 100 의 전설적인 소방관. 그의 특별한 능력은 긴장하는 순간, 미래가 보인...",네이버 웹툰,0.408163
431,굿 리스너,쥬드프라이데이,쥬드프라이데이,일,"옴니버스, 드라마",사무실을 공짜로 빌려쓰게 된 만화가 쥬드씨. 그 사무실은 원래 고민상담소로 쓰이던 ...,네이버 웹툰,0.188795
1411,구름의 노래,호랑,호랑,완결,"옴니버스, 드라마",삶을 풍요롭게 하는 음악 밴드를 통해 삶으로부터 탈출하고자 하는 그들.노래를 통해 ...,네이버 웹툰,0.187617
1124,오민혁 단편선,오민혁,오민혁,완결,"옴니버스, 드라마","한 명의 젊은 작가가 그리는, 때로는 기발하고 때로는 따뜻한 세계! 짧지만 그래서 ...",네이버 웹툰,0.185644
1263,아이들의 권 선생님,호우,호우,완결,"옴니버스, 드라마",아이들의 히어로! 조폭 선생님과 다섯 꼬마들의 소소한 학교 일상 이야기.,네이버 웹툰,0.184616
1408,싸우자귀신아,임인스,임인스,완결,"옴니버스, 드라마",귀신과 퇴마사의 좌충우돌 한판승부! 골 때리는 장르 불명의 이야기!,네이버 웹툰,0.184450


In [49]:
get_recomm('죽음에 관하여',noauthor=True,w_author=1.3, w_genre=0.7)

,title,writer,illustrator,day,genre,story,platform,similarity
431,굿 리스너,쥬드프라이데이,쥬드프라이데이,일,"옴니버스, 드라마",사무실을 공짜로 빌려쓰게 된 만화가 쥬드씨. 그 사무실은 원래 고민상담소로 쓰이던 ...,네이버 웹툰,0.511082
1411,구름의 노래,호랑,호랑,완결,"옴니버스, 드라마",삶을 풍요롭게 하는 음악 밴드를 통해 삶으로부터 탈출하고자 하는 그들.노래를 통해 ...,네이버 웹툰,0.507891
1124,오민혁 단편선,오민혁,오민혁,완결,"옴니버스, 드라마","한 명의 젊은 작가가 그리는, 때로는 기발하고 때로는 따뜻한 세계! 짧지만 그래서 ...",네이버 웹툰,0.502550
1263,아이들의 권 선생님,호우,호우,완결,"옴니버스, 드라마",아이들의 히어로! 조폭 선생님과 다섯 꼬마들의 소소한 학교 일상 이야기.,네이버 웹툰,0.499768
1408,싸우자귀신아,임인스,임인스,완결,"옴니버스, 드라마",귀신과 퇴마사의 좌충우돌 한판승부! 골 때리는 장르 불명의 이야기!,네이버 웹툰,0.499321
355,단편.zip,웹툰작가,웹툰작가,토,"옴니버스, 드라마","짧고 강렬하고 신선한 단편 모음 . 장르, 소재, 설정, 길이 제한 없음! 매주 새...",네이버 웹툰,0.499283
1164,차차차,한나,한나,완결,"옴니버스, 드라마",감성웹툰 묵회 한나 작가의 신작! 한가한 신생 찻집 고운 다실 에서 일어나는 따뜻하...,네이버 웹툰,0.499280
1255,고고루키루,한경찰,한경찰,완결,"옴니버스, 드라마","정체불명의 3인조, 천진난만 켄, 카리스마 선녀, 쉬크노노 루키루가 만들어내는 황당...",네이버 웹툰,0.498702
1435,사노라면,네이버,작가,완결,"옴니버스, 드라마",사노라면 힘든날도 있습니다.하지만 함께 하는 사람들이 있기에 다시 해가 뜰 날을 기...,네이버 웹툰,0.498693
1205,2014 루키 단편선-지옥...,웹툰작가,웹툰작가,완결,"옴니버스, 드라마","전설적인 작가 이현세, 이두호가 직접 프로듀싱한신인 작가 단편선 네번째 시즌!",네이버 웹툰,0.498608


### w_author=1.0, w_genre=1.3

In [50]:
get_recomm('신의 탑',w_author=1.0, w_genre=1.3)

,title,writer,illustrator,day,genre,story,platform,similarity
261,나 혼자 만렙 뉴비,메슬로우,스윙뱃,금,"스토리, 판타지","게임 너튜버로 활동 중인 주인공 진혁은 유일하게 시련의 탑 의 엔딩을 봤지만, 게임...",네이버 웹툰,0.542088
1441,창위의일루젼,시우,시우,완결,"스토리, 판타지",눈이 오는 겨울밤 소년을 쫓아 오는 악령의 기사...,네이버 웹툰,0.537485
89,빌런투킬,퓨핀,은지,화,"스토리, 판타지",동료를 살해했다는 억울한 누명을 쓰고 사망한 카시안. 모든 게 끝이라고 생각한 순간...,네이버 웹툰,0.536434
931,환상적인 소년,후은,후은,완결,"스토리, 판타지",평범한 소년과 환상적인 소녀의 위험한 동행 이야기.,네이버 웹툰,0.536348
715,골든 체인지,브림스,브림스,완결,"스토리, 판타지",세상에서 가장 불운한 소녀 강유와 소녀의 불운을 막아주는 수상한 산신령 우암이 펼치...,네이버 웹툰,0.535943
60,오로지 오로라,홍달,홍달,월,"스토리, 판타지","마법 소녀가 불법인 세상, 이 세계의 마법 소녀는 사랑받는 존재가 아니다. 한창 잘...",네이버 웹툰,0.535473
1179,붉은 실,동비,동비,완결,"스토리, 판타지",생명의 붉은 실을 두고 벌어지는 사건 그리고 거기에 휘말린 소년과 소녀의 이야기,네이버 웹툰,0.535042
1081,천년구미호,기량,기량,완결,"스토리, 판타지",세습무의 운명을 가진 소녀와 천년의 속박에서 벗어나기 위한 구미호이야기,네이버 웹툰,0.534733
569,리안소울의 엑소클럽,신진오,박희정,완결,"스토리, 판타지",녹안 을 가진 신비한 소녀 소울과 천재 사이코패스 리안. 미스테리한 저주계 사건을 ...,네이버 웹툰,0.534697
510,뫼신 사냥꾼,윤현승,방승현,완결,"스토리, 판타지","신령과도 같은 짐승 뫼신.떠돌이 무사 한세희는 그들을 사냥하고, 그의 앞에 신비한 ...",네이버 웹툰,0.534304


In [51]:
get_recomm('신의 탑',noauthor=True,w_author=1.0, w_genre=1.3)

,title,writer,illustrator,day,genre,story,platform,similarity
261,나 혼자 만렙 뉴비,메슬로우,스윙뱃,금,"스토리, 판타지","게임 너튜버로 활동 중인 주인공 진혁은 유일하게 시련의 탑 의 엔딩을 봤지만, 게임...",네이버 웹툰,0.789617
1441,창위의일루젼,시우,시우,완결,"스토리, 판타지",눈이 오는 겨울밤 소년을 쫓아 오는 악령의 기사...,네이버 웹툰,0.782912
89,빌런투킬,퓨핀,은지,화,"스토리, 판타지",동료를 살해했다는 억울한 누명을 쓰고 사망한 카시안. 모든 게 끝이라고 생각한 순간...,네이버 웹툰,0.781381
931,환상적인 소년,후은,후은,완결,"스토리, 판타지",평범한 소년과 환상적인 소녀의 위험한 동행 이야기.,네이버 웹툰,0.781256
715,골든 체인지,브림스,브림스,완결,"스토리, 판타지",세상에서 가장 불운한 소녀 강유와 소녀의 불운을 막아주는 수상한 산신령 우암이 펼치...,네이버 웹툰,0.780665
60,오로지 오로라,홍달,홍달,월,"스토리, 판타지","마법 소녀가 불법인 세상, 이 세계의 마법 소녀는 사랑받는 존재가 아니다. 한창 잘...",네이버 웹툰,0.779982
1179,붉은 실,동비,동비,완결,"스토리, 판타지",생명의 붉은 실을 두고 벌어지는 사건 그리고 거기에 휘말린 소년과 소녀의 이야기,네이버 웹툰,0.779354
1081,천년구미호,기량,기량,완결,"스토리, 판타지",세습무의 운명을 가진 소녀와 천년의 속박에서 벗어나기 위한 구미호이야기,네이버 웹툰,0.778903
569,리안소울의 엑소클럽,신진오,박희정,완결,"스토리, 판타지",녹안 을 가진 신비한 소녀 소울과 천재 사이코패스 리안. 미스테리한 저주계 사건을 ...,네이버 웹툰,0.778851
510,뫼신 사냥꾼,윤현승,방승현,완결,"스토리, 판타지","신령과도 같은 짐승 뫼신.떠돌이 무사 한세희는 그들을 사냥하고, 그의 앞에 신비한 ...",네이버 웹툰,0.778279


In [52]:
get_recomm('비흔',w_author=1.0, w_genre=1.3)

,title,writer,illustrator,day,genre,story,platform,similarity
1455,태왕광개토,청안랑,김인호,완결,"스토리, 시대극",고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.534319
1276,삵의 발톱,손영완,손영완,완결,"스토리, 시대극","중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.533819
1016,천국의 신화 6부,올댓스토리,이현세,완결,"스토리, 시대극","10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.532655
1387,흐드러지다,연제원,연제원,완결,"스토리, 시대극","지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.532473
763,조선방랑야사,가현,가현,완결,"스토리, 시대극",한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.531925
1177,문아,팬마,팬마,완결,"스토리, 시대극",평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.530971
138,화산귀환,비가,LICO,수,"스토리, 시대극",대 화산파 13대 제자.천하삼대검수 매화검존 청명. 천하를 혼란에 빠뜨린 고금제일마...,네이버 웹툰,0.530685
900,왕으로 살다,네스티캣,네스티캣,완결,"스토리, 시대극",살해당한 쌍둥이 오빠를 대신해 왕의 자리에 오른 화현의 운명은?,네이버 웹툰,0.530473
1325,한섬세대,유승진,유승진,완결,"스토리, 시대극","조선후기의 평범한 관리지망생 한섬. 만만치 않은 길이지만, 한섬이는 만만디. 과연 ...",네이버 웹툰,0.530307
1137,"아이소포스 1, 2부",김양수,도가도,완결,"스토리, 시대극","이솝우화의 이솝, 자신의 운명을 바꾼 한 사람의 이야기.",네이버 웹툰,0.530234


In [53]:
get_recomm('비흔',noauthor=True,w_author=1.0, w_genre=1.3)

,title,writer,illustrator,day,genre,story,platform,similarity
1455,태왕광개토,청안랑,김인호,완결,"스토리, 시대극",고구려 19대 왕! 한반도 역사상 최대의 영토를 확장한 정복 군주. 고구려의 세력권...,네이버 웹툰,0.778301
1276,삵의 발톱,손영완,손영완,완결,"스토리, 시대극","중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.777571
1016,천국의 신화 6부,올댓스토리,이현세,완결,"스토리, 시대극","10년을 기다린 이현세 화백의 천국의 신화, 그 다음이야기.새로운 대단원의 막이 오른다.",네이버 웹툰,0.775876
1387,흐드러지다,연제원,연제원,완결,"스토리, 시대극","지배하는 자, 차지하려는 자 그리고 암살자.쥬신의 황궁을 둘러 싼 음모와 배신그 뒤...",네이버 웹툰,0.775611
763,조선방랑야사,가현,가현,완결,"스토리, 시대극",한때 조선 한양 책방에서 연애담 소설로 최고 인기를 구가하던 선비 성기임.수험 압박...,네이버 웹툰,0.774813
1177,문아,팬마,팬마,완결,"스토리, 시대극",평범한 삶을 살려는 특별한 사람과 특별한 자신을 찾으려는 평범한 사람 신과 사람의 ...,네이버 웹툰,0.773424
138,화산귀환,비가,LICO,수,"스토리, 시대극",대 화산파 13대 제자.천하삼대검수 매화검존 청명. 천하를 혼란에 빠뜨린 고금제일마...,네이버 웹툰,0.773008
900,왕으로 살다,네스티캣,네스티캣,완결,"스토리, 시대극",살해당한 쌍둥이 오빠를 대신해 왕의 자리에 오른 화현의 운명은?,네이버 웹툰,0.772698
1325,한섬세대,유승진,유승진,완결,"스토리, 시대극","조선후기의 평범한 관리지망생 한섬. 만만치 않은 길이지만, 한섬이는 만만디. 과연 ...",네이버 웹툰,0.772457
1137,"아이소포스 1, 2부",김양수,도가도,완결,"스토리, 시대극","이솝우화의 이솝, 자신의 운명을 바꾼 한 사람의 이야기.",네이버 웹툰,0.772351


In [54]:
get_recomm('마음의소리',w_author=1.0, w_genre=1.3)

,title,writer,illustrator,day,genre,story,platform,similarity
1454,취업의소리,조석,워니,완결,"에피소드, 개그","조석, 박종원 2인의 작가들이 펼치는,팍팍!! 막힘 없이 뚫리는 취업이야기. 웃음은...",네이버 웹툰,0.686520
1157,조석축구만화,조석,조석,완결,"에피소드, 스포츠",조석 작가가 들려주는 너무나도 솔직담백한 해외축구이야기.,네이버 웹툰,0.595613
129,피로만땅,샤니,샤니,화,"에피소드, 개그",203호 저승사자 샤니 작가 신작. 취준생 유림은 밤 9시에 면접을 보러오라는 연락...,네이버 웹툰,0.533128
22,오늘의 순정망화,손하기,손하기,"월, 화, 목","에피소드, 개그",그마고 초엘리트집단 삼국시대 앞에 나타난 전학생 가야! 가야의 마음을 얻는 건 누구?,네이버 웹툰,0.531772
946,조선왕조실톡,무적핑크,무적핑크,완결,"에피소드, 개그","어느 날 메신저로 찾아온, 조선 시대 그분들의 시시콜콜한 이야기.무적핑크 작가 신작!",네이버 웹툰,0.531285
772,203호 저승사자,샤니,샤니,완결,"에피소드, 개그",어느날 저승사자가 찾아왔다.그런데 이 저승사자... 제 정신이 아니다.,네이버 웹툰,0.530825
1457,트라우마,곽백수,곽백수,완결,"에피소드, 개그",황당하고 골 때리는 반전! 정신적 외상 를 입을 만한 초특급 코믹 펀치! 직접 확인하세요,네이버 웹툰,0.530635
1085,밥풀때기,Dylan,Dylan,완결,"에피소드, 개그",열정이 가득한 밥풀때기 아저씨의 다이내믹한 하루! 글로벌 라인 웹툰에서 연재되고 있...,네이버 웹툰,0.530491
841,백귀야행지,아만,아만,완결,"에피소드, 개그",소름과 웃음이 동시에! 빨간 도깨비가 풀어놓는 기이한 이야기글로벌 라인 웹툰에서 연...,네이버 웹툰,0.530466
903,놓지마 정신줄 시즌2,신태훈,나승훈,완결,"에피소드, 개그","정신, 정주리 남매와 독특한 주변 인물들의정신줄 놓게 만드는 다양한 에피소드들!본격...",네이버 웹툰,0.530429


In [55]:
get_recomm('마음의소리', noauthor=True,w_author=1.0, w_genre=1.3)

,title,writer,illustrator,day,genre,story,platform,similarity
129,피로만땅,샤니,샤니,화,"에피소드, 개그",203호 저승사자 샤니 작가 신작. 취준생 유림은 밤 9시에 면접을 보러오라는 연락...,네이버 웹툰,0.776566
22,오늘의 순정망화,손하기,손하기,"월, 화, 목","에피소드, 개그",그마고 초엘리트집단 삼국시대 앞에 나타난 전학생 가야! 가야의 마음을 얻는 건 누구?,네이버 웹툰,0.774590
946,조선왕조실톡,무적핑크,무적핑크,완결,"에피소드, 개그","어느 날 메신저로 찾아온, 조선 시대 그분들의 시시콜콜한 이야기.무적핑크 작가 신작!",네이버 웹툰,0.773880
772,203호 저승사자,샤니,샤니,완결,"에피소드, 개그",어느날 저승사자가 찾아왔다.그런데 이 저승사자... 제 정신이 아니다.,네이버 웹툰,0.773211
1457,트라우마,곽백수,곽백수,완결,"에피소드, 개그",황당하고 골 때리는 반전! 정신적 외상 를 입을 만한 초특급 코믹 펀치! 직접 확인하세요,네이버 웹툰,0.772933
1085,밥풀때기,Dylan,Dylan,완결,"에피소드, 개그",열정이 가득한 밥풀때기 아저씨의 다이내믹한 하루! 글로벌 라인 웹툰에서 연재되고 있...,네이버 웹툰,0.772725
841,백귀야행지,아만,아만,완결,"에피소드, 개그",소름과 웃음이 동시에! 빨간 도깨비가 풀어놓는 기이한 이야기글로벌 라인 웹툰에서 연...,네이버 웹툰,0.772688
903,놓지마 정신줄 시즌2,신태훈,나승훈,완결,"에피소드, 개그","정신, 정주리 남매와 독특한 주변 인물들의정신줄 놓게 만드는 다양한 에피소드들!본격...",네이버 웹툰,0.772635
124,고등매직,잭형,잭형,화,"에피소드, 개그",평범한 고등학생인 마삼룡 의 집에 갑자기 삼촌이 찾아온다. 삼룡의 가족과 함께 지내...,네이버 웹툰,0.772327
1254,내일은 웹툰,신의철,신의철,완결,"에피소드, 개그","내일 나오는 만화를 그리며, 언제나 내일을 사는 웹툰 작가들의 만화같은 이야기.",네이버 웹툰,0.772024


In [56]:
get_recomm('죽음에 관하여',w_author=1.0, w_genre=1.3)

,title,writer,illustrator,day,genre,story,platform,similarity
1233,네가 없는 세상,시니,혀노,완결,"스토리, 드라마",죽음에 관하여 시니 혀노 작가 신작. 나 만 아는 세상.,네이버 웹툰,0.581112
431,굿 리스너,쥬드프라이데이,쥬드프라이데이,일,"옴니버스, 드라마",사무실을 공짜로 빌려쓰게 된 만화가 쥬드씨. 그 사무실은 원래 고민상담소로 쓰이던 ...,네이버 웹툰,0.534787
1411,구름의 노래,호랑,호랑,완결,"옴니버스, 드라마",삶을 풍요롭게 하는 음악 밴드를 통해 삶으로부터 탈출하고자 하는 그들.노래를 통해 ...,네이버 웹툰,0.533797
1124,오민혁 단편선,오민혁,오민혁,완결,"옴니버스, 드라마","한 명의 젊은 작가가 그리는, 때로는 기발하고 때로는 따뜻한 세계! 짧지만 그래서 ...",네이버 웹툰,0.532139
1263,아이들의 권 선생님,호우,호우,완결,"옴니버스, 드라마",아이들의 히어로! 조폭 선생님과 다섯 꼬마들의 소소한 학교 일상 이야기.,네이버 웹툰,0.531276
1408,싸우자귀신아,임인스,임인스,완결,"옴니버스, 드라마",귀신과 퇴마사의 좌충우돌 한판승부! 골 때리는 장르 불명의 이야기!,네이버 웹툰,0.531137
355,단편.zip,웹툰작가,웹툰작가,토,"옴니버스, 드라마","짧고 강렬하고 신선한 단편 모음 . 장르, 소재, 설정, 길이 제한 없음! 매주 새...",네이버 웹툰,0.531126
1164,차차차,한나,한나,완결,"옴니버스, 드라마",감성웹툰 묵회 한나 작가의 신작! 한가한 신생 찻집 고운 다실 에서 일어나는 따뜻하...,네이버 웹툰,0.531125
1255,고고루키루,한경찰,한경찰,완결,"옴니버스, 드라마","정체불명의 3인조, 천진난만 켄, 카리스마 선녀, 쉬크노노 루키루가 만들어내는 황당...",네이버 웹툰,0.530945
1435,사노라면,네이버,작가,완결,"옴니버스, 드라마",사노라면 힘든날도 있습니다.하지만 함께 하는 사람들이 있기에 다시 해가 뜰 날을 기...,네이버 웹툰,0.530942


In [57]:
get_recomm('죽음에 관하여',noauthor=True,w_author=1.0, w_genre=1.3)

,title,writer,illustrator,day,genre,story,platform,similarity
431,굿 리스너,쥬드프라이데이,쥬드프라이데이,일,"옴니버스, 드라마",사무실을 공짜로 빌려쓰게 된 만화가 쥬드씨. 그 사무실은 원래 고민상담소로 쓰이던 ...,네이버 웹툰,0.778982
1411,구름의 노래,호랑,호랑,완결,"옴니버스, 드라마",삶을 풍요롭게 하는 음악 밴드를 통해 삶으로부터 탈출하고자 하는 그들.노래를 통해 ...,네이버 웹툰,0.777540
1124,오민혁 단편선,오민혁,오민혁,완결,"옴니버스, 드라마","한 명의 젊은 작가가 그리는, 때로는 기발하고 때로는 따뜻한 세계! 짧지만 그래서 ...",네이버 웹툰,0.775125
1263,아이들의 권 선생님,호우,호우,완결,"옴니버스, 드라마",아이들의 히어로! 조폭 선생님과 다섯 꼬마들의 소소한 학교 일상 이야기.,네이버 웹툰,0.773868
1408,싸우자귀신아,임인스,임인스,완결,"옴니버스, 드라마",귀신과 퇴마사의 좌충우돌 한판승부! 골 때리는 장르 불명의 이야기!,네이버 웹툰,0.773665
355,단편.zip,웹툰작가,웹툰작가,토,"옴니버스, 드라마","짧고 강렬하고 신선한 단편 모음 . 장르, 소재, 설정, 길이 제한 없음! 매주 새...",네이버 웹툰,0.773649
1164,차차차,한나,한나,완결,"옴니버스, 드라마",감성웹툰 묵회 한나 작가의 신작! 한가한 신생 찻집 고운 다실 에서 일어나는 따뜻하...,네이버 웹툰,0.773647
1255,고고루키루,한경찰,한경찰,완결,"옴니버스, 드라마","정체불명의 3인조, 천진난만 켄, 카리스마 선녀, 쉬크노노 루키루가 만들어내는 황당...",네이버 웹툰,0.773386
1435,사노라면,네이버,작가,완결,"옴니버스, 드라마",사노라면 힘든날도 있습니다.하지만 함께 하는 사람들이 있기에 다시 해가 뜰 날을 기...,네이버 웹툰,0.773382
1205,2014 루키 단편선-지옥...,웹툰작가,웹툰작가,완결,"옴니버스, 드라마","전설적인 작가 이현세, 이두호가 직접 프로듀싱한신인 작가 단편선 네번째 시즌!",네이버 웹툰,0.773343


In [58]:
get_recomm('뽀짜툰',w_author=1.0, w_genre=1.3)

,title,writer,illustrator,day,genre,story,platform,similarity
1527,금수친구들,짤태식,짤태식,수,코믹/일상,"평범한 나무꾼, 금수 네 마리의 집사가 되다! 고요한 아침의 나라 조선, 어느 작은...",카카오웹툰,0.538291
1730,냥냐라냥냥,손달섭,손달섭,완결,코믹/일상,나만 없어 고양이 집사! 갑자기 생긴 고양이 집사들과의 좌충우돌 휴먼 다큐멘터리,카카오웹툰,0.537010
2167,쏘쏘리,남지은,남지은,완결,코믹/일상,부족한 것 투성이로 살아가는 30대 싱글녀 쏘쏘리와 그녀의 주변인들의 삶 이야기!,카카오웹툰,0.536771
2052,상상고양이,김경,김경,완결,코믹/일상,"인간과 고양이... 같은 상황, 다른 생각! 두둥!",카카오웹툰,0.536094
2130,그루밍 선데이,Hun,Hun,완결,코믹/일상,Hun 작가와 작은 고양이 알콩달콩 동거 이야기,카카오웹툰,0.535940
2099,탐묘인간,soon,soon,완결,코믹/일상,고양이의 모든 것이 사랑스러워.. 당신도 나와 같은가요?,카카오웹툰,0.534728
1818,남은 고양이,김경,김경,완결,코믹/일상,남은 고양이의 임무는 내 인간이 나 없이도 튼튼하게 지내도록 만드는 것!,카카오웹툰,0.534052
2050,에이스 하이,이창현,이창현,완결,코믹/일상,4년 만에 돌아온 J.J. 이번에는 출격할 것인가,카카오웹툰,0.533915
1591,퀴퀴한 일기,2B,2B,금,코믹/일상,"내 주변에 반드시 한 명은 있을 것만 같은, 어쩌면 나 같기도 한 그녀의 일기. 어...",카카오웹툰,0.533605
2006,아빠는 16세,서랍천사,서랍천사,완결,코믹/일상,"내게 완벽한 삶을 주겠다던 외계인, 그런데 뭔가 이상하다..",카카오웹툰,0.533586


In [65]:
get_recomm('삼국지톡',w_author=1.3, w_genre=1.0)

,title,writer,illustrator,day,genre,story,platform,similarity
946,조선왕조실톡,무적핑크,무적핑크,완결,"에피소드, 개그","어느 날 메신저로 찾아온, 조선 시대 그분들의 시시콜콜한 이야기.무적핑크 작가 신작!",네이버 웹툰,0.428548
1329,실질객관동화,무적핑크,무적핑크,완결,"에피소드, 개그",동화를 객관적인 시각에서 보면 어떻게 될까?생소한 시각으로 동화 다시 보기,네이버 웹툰,0.422339
1230,실질객관영화,무적핑크,무적핑크,완결,"에피소드, 개그",10층에서 뛰어내렸는데 어떻게 살아있을 수 있지?정말 히어로는 무조건 이길 수 있을...,네이버 웹툰,0.422066
1296,경운기를 탄 왕자님,무적핑크,무적핑크,완결,"스토리, 개그","실질객관동화 무적핑크 작가의 신작!싹수 노란 남자의 농사 정복기, 오빤 강남 농부 ...",네이버 웹툰,0.265119
1466,까뱅,X,TEAM,완결,"에피소드, 개그",치킨시티의 새로운 영웅 까뱅 오늘도 그의 활약은 계속된다!,네이버 웹툰,0.167166
1276,삵의 발톱,손영완,손영완,완결,"스토리, 시대극","중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.164651
895,데우스 엑스 마키나,꼬마비,꼬마비,완결,"에피소드, 드라마",신은 어디에나 있다,네이버 웹툰,0.163713
352,웰캄투실버라이프,솔녀,솔녀,토,"에피소드, 일상","보고만 있어도 힐링되는 할아버지, 할머니와의 일상! 라떼는 말이야 금지!! 황혼 청...",네이버 웹툰,0.161581
1458,식스센스,릴레이연재,릴레이연재,완결,"에피소드, 개그",매주 새로운 주제의 6인 6색 만화를 즐기자!,네이버 웹툰,0.161503
653,회춘,기안84,기안84,완결,"에피소드, 드라마",두 번의 청춘이 오는 세상에서살아가는 사람들 이야기.기안84 작가의 새로운 작품!,네이버 웹툰,0.160834


In [66]:
get_recomm('삼국지톡')

,title,writer,illustrator,day,genre,story,platform,similarity
946,조선왕조실톡,무적핑크,무적핑크,완결,"에피소드, 개그","어느 날 메신저로 찾아온, 조선 시대 그분들의 시시콜콜한 이야기.무적핑크 작가 신작!",네이버 웹툰,0.414951
1329,실질객관동화,무적핑크,무적핑크,완결,"에피소드, 개그",동화를 객관적인 시각에서 보면 어떻게 될까?생소한 시각으로 동화 다시 보기,네이버 웹툰,0.407560
1230,실질객관영화,무적핑크,무적핑크,완결,"에피소드, 개그",10층에서 뛰어내렸는데 어떻게 살아있을 수 있지?정말 히어로는 무조건 이길 수 있을...,네이버 웹툰,0.407236
895,데우스 엑스 마키나,꼬마비,꼬마비,완결,"에피소드, 드라마",신은 어디에나 있다,네이버 웹툰,0.332445
1466,까뱅,X,TEAM,완결,"에피소드, 개그",치킨시티의 새로운 영웅 까뱅 오늘도 그의 활약은 계속된다!,네이버 웹툰,0.327709
1276,삵의 발톱,손영완,손영완,완결,"스토리, 시대극","중국의 위촉오시대, 삼국지연의만이 전부가 아니다. 고구려인, 백제인의 시각에서 바라...",네이버 웹툰,0.324715
352,웰캄투실버라이프,솔녀,솔녀,토,"에피소드, 일상","보고만 있어도 힐링되는 할아버지, 할머니와의 일상! 라떼는 말이야 금지!! 황혼 청...",네이버 웹툰,0.321060
1458,식스센스,릴레이연재,릴레이연재,완결,"에피소드, 개그",매주 새로운 주제의 6인 6색 만화를 즐기자!,네이버 웹툰,0.320968
653,회춘,기안84,기안84,완결,"에피소드, 드라마",두 번의 청춘이 오는 세상에서살아가는 사람들 이야기.기안84 작가의 새로운 작품!,네이버 웹툰,0.320172
516,인터셉트,임주이,임주이,완결,"에피소드, 로맨스",비 오는 날이면 늘 안 좋은 일만 생기던 그녀에게 처음으로 내린 따스한 비.사랑에 ...,네이버 웹툰,0.319809


In [67]:
get_recomm('조선왕조실톡')

,title,writer,illustrator,day,genre,story,platform,similarity
1230,실질객관영화,무적핑크,무적핑크,완결,"에피소드, 개그",10층에서 뛰어내렸는데 어떻게 살아있을 수 있지?정말 히어로는 무조건 이길 수 있을...,네이버 웹툰,0.814830
1329,실질객관동화,무적핑크,무적핑크,완결,"에피소드, 개그",동화를 객관적인 시각에서 보면 어떻게 될까?생소한 시각으로 동화 다시 보기,네이버 웹툰,0.814226
1031,오늘도 형제는 평화롭다,GIMS,GIMS,완결,"에피소드, 개그","돈독한 형제들의 이야기, 사건의 중심엔 늘 그들이 있다.",네이버 웹툰,0.635236
1206,소년들은 무엇을 하고 있을까,컷부,컷부,완결,"에피소드, 개그",소년들 사이에서 벌어지는 다양한 이야기를 다룹니다.,네이버 웹툰,0.635083
772,203호 저승사자,샤니,샤니,완결,"에피소드, 개그",어느날 저승사자가 찾아왔다.그런데 이 저승사자... 제 정신이 아니다.,네이버 웹툰,0.634811
864,가우스전자 시즌3~4,곽백수,곽백수,완결,"에피소드, 개그",다국적 문어발 기업 가우스전자에서 벌어지는웃픈 현실 직장인 이야기,네이버 웹툰,0.633785
1254,내일은 웹툰,신의철,신의철,완결,"에피소드, 개그","내일 나오는 만화를 그리며, 언제나 내일을 사는 웹툰 작가들의 만화같은 이야기.",네이버 웹툰,0.633618
124,고등매직,잭형,잭형,화,"에피소드, 개그",평범한 고등학생인 마삼룡 의 집에 갑자기 삼촌이 찾아온다. 삼룡의 가족과 함께 지내...,네이버 웹툰,0.633282
22,오늘의 순정망화,손하기,손하기,"월, 화, 목","에피소드, 개그",그마고 초엘리트집단 삼국시대 앞에 나타난 전학생 가야! 가야의 마음을 얻는 건 누구?,네이버 웹툰,0.633027
841,백귀야행지,아만,아만,완결,"에피소드, 개그",소름과 웃음이 동시에! 빨간 도깨비가 풀어놓는 기이한 이야기글로벌 라인 웹툰에서 연...,네이버 웹툰,0.632846
